# Assignment: RAG Workflow with LangChain, Groq API, and FAISS

In this assignment, you will:

1. Load web content using LangChain.
2. Store embeddings of the content in FAISS, a vector database.
3. Use Groq API as your LLM to query the stored embeddings via Retrieval-Augmented Generation (RAG).

## Objectives:

- Learn how to use LangChain for data loading from the web.
- Understand embeddings and how to store/retrieve them with FAISS.
- Implement RAG workflow using Groq LLM API.

## Setup and Installation


In [1]:
# Install required libraries
!pip install langchain groq faiss-cpu sentence-transformers beautifulsoup4 langchain-community langchain-groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.6/129.6 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 25.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 34.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 31.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 42.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.

## Import Libraries

In [2]:
from langchain.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain_groq import ChatGroq
import os

## Step 1: Load and Prepare Data

https://python.langchain.com/docs/integrations/document_loaders/web_base/

https://python.langchain.com/api_reference/text_splitters/character/langchain_text_splitters.character.RecursiveCharacterTextSplitter.html

https://huggingface.co/blog/getting-started-with-embeddings

https://python.langchain.com/docs/integrations/vectorstores/faiss/

# Documentation

In [8]:
from googlesearch import search
query = "LangChain"
num_results = 5
def get_top_urls(query, num_results=5):
    return list(search(query, stop=num_results, pause=2))
# url = "https://en.wikipedia.org/wiki/LangChain"  # Example URL, replace with your choice
urls = get_top_urls(query,num_results)
loader = WebBaseLoader(urls)
docs = loader.load()

In [9]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
documents = text_splitter.split_documents(docs)

## Step 2: Generate Embeddings and Store in FAISS

In [10]:
embeddings_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

<ipython-input-10-9ad8b13905eb>:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warni

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [11]:
vectorstore = FAISS.from_documents(documents, embeddings_model)

## Step 3: Setup Groq LLM

In [12]:
from google.colab import userdata
os.environ['GROQ_API_KEY'] = userdata.get('groq')

In [13]:
llm = ChatGroq(model_name="llama3-8b-8192")

## Step 4: Retrieval-Augmented Generation (RAG)

In [14]:
retriever = vectorstore.as_retriever()
qa_chain = RetrievalQA.from_chain_type(llm=llm, retriever=retriever)

## Step 5: Querying the System

In [15]:
query = "What is LangChain and how does it work?"
answer = qa_chain.run(query)

<ipython-input-15-108edd1eb519>:2: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  answer = qa_chain.run(query)


In [16]:
print(f"Question: {query}\n")
print(f"Answer: {answer}\n")

Question: What is LangChain and how does it work?

Answer: According to the provided context, LangChain is a software framework that helps facilitate the integration of large language models (LLMs) into applications. It is a language model integration framework that can be used standalone or integrates seamlessly with LangChain products to provide a full suite of tools for building LLM applications.

LangChain was launched in October 2022 as an open-source project and has gained popularity with contributions from hundreds of developers, discussions on social media, and meetups. The framework is designed to work with other LangChain products, which can be used independently or stacked for multiplicative benefits.

LangChain's products work together to provide an integrated solution for every step of the application development journey, covering document analysis and summarization, chatbots, and code analysis.

